In [77]:
import pandas as pd
pd.set_option("display.max_columns",10) #設定pandas最多顯示出50個欄位資訊
pd.set_option('display.max_rows', None)

df =  pd.read_table("testing_label.txt")
df.rename(columns={"0#####don ' t you hate it when you hang on to a seemingly interesting movie to see the ending only to find out that the ending sucks ?":'文字'}, inplace = True)
new=pd.DataFrame({'文字':"0#####don ' t you hate it when you hang on to a seemingly interesting movie to see the ending only to find out that the ending sucks ?",
                  },
                 index=[1]) 
df=df.append(new,ignore_index=True)   # ignore_index=True,表示不按原来的索引，从0开始自动递增

In [78]:
df['變數']=df['文字'].str.split("#####").str.get(0).str.title()
df['字串']=df['文字'].str.split("#####").str.get(1).str.title()
df.head()

,文字,變數,字串
0,0#####ok so never went to the movies because f...,0,Ok So Never Went To The Movies Because Friend ...
1,1#####can ' t wait to see diversity ' s perfor...,1,Can ' T Wait To See Diversity ' S Performance !
2,1#####i love britney spears haha joey this is ...,1,I Love Britney Spears Haha Joey This Is What U...
3,1#####1 more day !,1,1 More Day !
4,0#####hates being this burnt !! ouch,0,Hates Being This Burnt !! Ouch


In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer2 = TfidfVectorizer()
X_test = vectorizer2.fit_transform(df['字串'])
Y_test = df['變數']
#print(vectorizer.get_feature_names())
print(X_test.shape)

(90, 519)


In [80]:
df1 =  pd.read_table("training_label.txt")

df1.rename(columns={"1 +++$+++ are wtf ... awww thanks !":'文字'}, inplace = True)
new=pd.DataFrame({'文字':"1 +++$+++ are wtf ... awww thanks !",
                  },
                 index=[1]) 
df1=df1.append(new,ignore_index=True)
train =df1.sample(10000)
training=train.reset_index(drop=True)
training.head()


,文字
0,1 +++$+++ i ' m at the gym ! good morning !
1,1 +++$+++ if you can think of a song to draw p...
2,"0 +++$+++ woohoohoo , missing my cousin noah ...."
3,"1 +++$+++ well , if you change your mind , i g..."
4,1 +++$+++ i would far rather mow the lawn than...


In [81]:
training['變數']=training['文字'].str.split('+').str.get(0).str.title()
training['字串']=training['文字'].str.split('+').str.get(6).str.title()
y=training['變數']

Y_test = df['變數']
print(X_test.shape)
print(X[:,:519].shape)
print(len(y))
Y_test=Y_test.astype(np.float64)
y=y.astype(np.float64)
print(Y_test.dtype)
print(X_test.dtype)
print(y.dtype)
training.head()

(90, 519)
(10000, 519)
10000
float64
float64
float64


,文字,變數,字串
0,1 +++$+++ i ' m at the gym ! good morning !,1,I ' M At The Gym ! Good Morning !
1,1 +++$+++ if you can think of a song to draw p...,1,If You Can Think Of A Song To Draw Pirates By...
2,"0 +++$+++ woohoohoo , missing my cousin noah ....",0,"Woohoohoo , Missing My Cousin Noah . Noah , N..."
3,"1 +++$+++ well , if you change your mind , i g...",1,"Well , If You Change Your Mind , I Got Some D..."
4,1 +++$+++ i would far rather mow the lawn than...,1,I Would Far Rather Mow The Lawn Than Shovel ....


In [57]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
vectorizer = CountVectorizer(analyzer='word', ngram_range=(3,3))
X = vectorizer.fit_transform(training['字串'])
X_test = vectorizer2.fit_transform(df['字串'])

(90, 519)
(10000, 519)
10000
float64
float64
float64


In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(training['字串'])
#print(vectorizer.get_feature_names())
print(X.shape)

(10000, 13567)


In [133]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
token = Tokenizer(num_words=3800) 
token.fit_on_texts(training['字串'])  
token.word_index
x_train_seq = token.texts_to_sequences(training['字串'])
x_test_seq = token.texts_to_sequences(df['字串'])
x_train = sequence.pad_sequences(x_train_seq, maxlen=380)
x_test = sequence.pad_sequences(x_test_seq, maxlen=380)
print(x_train.shape)
print(x_test.shape)

(10000, 380)
(90, 380)


In [108]:
 from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
adb = AdaBoostClassifier(n_estimators=400,learning_rate=0.8)
#adb.fit(X[:,:519], y)
adb.fit(x_train, y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.8,
                   n_estimators=400, random_state=None)

In [109]:
#adb.score(X[:,:519], y)
adb.score(x_train, y)

0.6257

In [110]:
from sklearn import metrics
print(metrics.classification_report(Y_test, adb.predict(x_test)))
#print(metrics.classification_report(y, adb.predict(x_train)))

              precision    recall  f1-score   support

         0.0       0.46      0.70      0.56        37
         1.0       0.68      0.43      0.53        53

    accuracy                           0.54        90
   macro avg       0.57      0.57      0.54        90
weighted avg       0.59      0.54      0.54        90



In [ ]:
import xgboost as xgb
xgbc = xgb.XGBClassifier(max_depth=10)
y=y.astype(np.int32)
#xgbr.fit(X[:,:519], y)
xgbc.fit(x_train, y)

In [129]:
xgbc.score(x_train, y)

0.9496

In [128]:
print(metrics.classification_report(Y_test, xgbc.predict(x_test)))
#print(metrics.classification_report(Y_test, xgbc.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.43      0.57      0.49        37
         1.0       0.61      0.47      0.53        53

    accuracy                           0.51        90
   macro avg       0.52      0.52      0.51        90
weighted avg       0.54      0.51      0.51        90

